In [10]:
!pip install caer canaro

In [11]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [12]:
IMG_SIZE = (80,80)

# channel 1 is grayscale
channels = 1
char_path = r"../input/the-simpsons-characters-dataset/simpsons_dataset"

In [13]:
# get all names of each face and the count for pictures
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))

# sort dictionary in descending order
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

In [14]:
# get the most popular characters I think
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break

characters

In [15]:
# create training data
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [19]:
len(train)

In [17]:
# show a random image
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [28]:
# seperate trainng set to geature set and labels
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [29]:
featureSet = np.array(featureSet)
labels = np.array(labels)

In [30]:
from tensorflow.keras.utils import to_categorical
# normalize featureSet ==> (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [35]:
# split trian and test data
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [36]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
# y_train.shape()

In [ ]:
"""# garbage collection
del train
del featureSet
del labels
gc.collect()"""

In [37]:
BATCH_SIZE = 32
EPOCHS = 10

In [38]:
# data generator for images
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

# Create our model!!!

In [40]:
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters), 
                                          loss='binary_crossentropy', decay=1e-6, learning_rate=0.001, 
                                          momentum=0.9, nesterov=True)

In [41]:
model.summary()

In [42]:
from tensorflow.keras.callbacks import LearningRateScheduler
# create a callbacks list
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [43]:
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

In [44]:
characters

In [ ]:
# resize all test images to be same as training set 
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [51]:
# take a random image to see the size
test_path = r"../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_27.jpg"
img = cv.imread(test_path)
plt.imshow(img, cmap='gray')
plt.show()

In [54]:
# predict which simpson character is in the image
predictions = model.predict(prepare(img))

In [61]:
print(characters[np.argmax(predictions[0])])